In [74]:
# import packages
import os
import csv
import random
import multiprocessing as mp

import tqdm
import requests

import numpy as np
import sklearn.model_selection as skms

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import pandas as pd

# Building Attributes Only Dataset

Be sure to have "image_class_labels.txt" and "image_attribute_labels.txt" in the same directory as this notebook

In [75]:
# Attributes Only Dataset 
class BirdAttributesDataset(Dataset):
    def __init__(self, attributes, labels):
        self.labels = labels
        self.attributes = attributes
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        attribute = self.attributes[idx]
        #sample = {"Attribute": attribute, "Label": label}
        #return sample
        return attribute, label

In [76]:
# Load Attributes Data and Labels
attr_data = np.loadtxt("image_attribute_labels.txt", delimiter=' ',dtype=float, usecols=(0,1,2))
image_id_attr = attr_data[:,0]
attributes_id = attr_data[:,1]
existences = attr_data[:,2]

img_data = np.loadtxt("image_class_labels.txt", delimiter=' ', dtype=float, usecols=(0,1))
image_id_label = img_data[:,0]
class_labels = img_data[:,1]


In [77]:
# Combining into a useable dataset
# Give each attribute data the label of its class
img_existence_data = list()
prev_label = image_id_attr[0]
curr_existence = list()
for i in range(len(existences)):
    this_label = image_id_attr[i]
    # Check to see if we are done with an image_id
    if(this_label != prev_label):
        img_existence_data.append(curr_existence)
        curr_existence = list()
        curr_existence.append(existences[i])
    else:
        curr_existence.append(existences[i])
    prev_label = this_label     
# Appending last values
img_existence_data.append(curr_existence)

In [78]:
attributes = img_existence_data
labels = class_labels

In [79]:
# Find top 10 classes
class_freq = np.zeros(200)

for i in range(len(labels)):
    class_freq[int(labels[i])-1] += 1
    
sorted_classes = np.argsort(class_freq)
top_10_classes = sorted_classes[-11:-1]


In [80]:
# Keep only top 10 classes
labels_top_10 = []
attributes_top_10 = []

for i in range(len(labels)):
    if labels[i] in top_10_classes:
        labels_top_10.append(labels[i])
        attributes_top_10.append(attributes[i])


In [81]:
print(top_10_classes)
print(len(labels_top_10))
print(len(attributes_top_10))

labels_normalized = []
# normalize labels
curr_label = labels_top_10[0]
curr_norm = 1
for i in range(len(labels_top_10)):
    if (labels_top_10[i] != curr_label):
        # new label
        curr_norm += 1
        curr_label = labels_top_10[i]
        labels_normalized.append(curr_norm)
    else:
        # same label
        labels_normalized.append(curr_norm)
        
        
labels = labels_normalized
attributes = attributes_top_10

print(labels)

[78 79 80 81 82 84 85 86 87 89]
592
592
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,

In [82]:
labels_df = pd.DataFrame({'Attributes': attributes, 'Labels': labels})
AD = BirdAttributesDataset(labels_df['Attributes'], labels_df['Labels'])

In [83]:
from sklearn.model_selection import train_test_split
train_indices, test_indices = train_test_split(list(range(len(AD))), test_size=0.3, stratify=labels)

In [84]:
train_data = torch.utils.data.Subset(AD, train_indices)
test_data = torch.utils.data.Subset(AD, test_indices)

In [85]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=False)

# Training and  Model

In [86]:
from tqdm import tqdm

In [87]:
class Net(nn.Module):
    def __init__(self, attribute_dim, n_classes):
        super(Net, self).__init__()
        self.hidden = nn.Linear(attribute_dim, 8)
        self.output = nn.Linear(8, n_classes)
        self.double()

    def forward(self, attributes):
        attributes = torch.sigmoid(self.hidden(attributes))
        output = self.output(attributes)
        return output

In [88]:
NUM_EPOCHS = 10
attribute_dim = len(attributes[0])
n_classes = 11

In [89]:
def train(net, trainloader, optimizer, loss_fn, num_epochs=NUM_EPOCHS):
    # Put the network in training mode
    net.train()

    # Training loop
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        t = tqdm(trainloader, desc='Epoch: 0, Loss: 0.0', position = 0, leave=True)
        
        for i, data in enumerate(trainloader):
            # get the inputs
            inputs, labels = data
            #print(len(inputs))
            #print(inputs[0])
            #print(len(inputs[0]))
            inputs = torch.transpose(torch.stack(inputs),0,1)
            optimizer.zero_grad() # zeroes gradients
            
            outputs = net.forward(inputs) # forward pass
            #print(labels)
            labels = labels.type(torch.LongTensor)
            loss = loss_fn(outputs, labels)
            loss.backward() # backward pass
            optimizer.step() # weight updates
            running_loss += loss.item()
            
            t.set_description(f'Epoch: {epoch+1}/{NUM_EPOCHS}, Loss: {running_loss/(i+1):.4f}', refresh=True)
            t.update()
            t.refresh()
            
            
    print('Finished Training')

In [90]:
def eval(net, data_loader, criterion, name=''):
    # set the network to evaluation mode
    net.eval()
    accuracy = 0
    loss = 0
    num_samples = 0
    with torch.no_grad():
        for data in tqdm(data_loader):
            inputs, labels = data
            inputs = torch.transpose(torch.stack(inputs),0,1)
            labels = labels.type(torch.LongTensor)
            out = net(inputs)
            loss_fn = criterion(out, labels)
            loss += loss_fn.item()
            accuracy += (out.argmax(dim=1) == labels).sum()
            num_samples += inputs.shape[0]
    loss = loss/num_samples
    accuracy = accuracy/num_samples
    print("Test Loss: {:.6f} \tAccuracy: {:.6f}".format(loss, accuracy))
    return loss, accuracy

In [91]:
NUM_EPOCHS = 100

In [92]:
net = Net(attribute_dim, n_classes)
LR = 0.0005
optimizer = torch.optim.AdamW(net.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
train(net, train_loader, optimizer, criterion, NUM_EPOCHS)

Epoch: 68/100, Loss: 0.9336: 100%|████████████████████████████████████████████████████| 52/52 [00:00<00:00, 260.01it/s]


Epoch: 100/100, Loss: 0.5988: 100%|███████████████████████████████████████████████████| 52/52 [00:00<00:00, 272.33it/s]

Finished Training


In [93]:
# Evaluating Net
test_loss, accuracy = eval(net, test_loader , criterion, 'Attributes test')


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 574.88it/s]

Test Loss: 0.105567 	Accuracy: 0.780899
